<a href="https://colab.research.google.com/github/Diegobr72/Colab/blob/main/Reconstru%C3%A7%C3%A3o_de_Informa%C3%A7%C3%B5es_Regress%C3%A3o_Linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reconstrução de Informações - Regressão Linear

**Vence:** 22/09/2024 às 23:59  
**Curso:** MPLS  
**Valor:** 0,75 pontos no Checkpoint 2

## Instruções

Utilizando a base de dados `gamba.csv`:




 ## 1) Crie um modelo preditivo para determinar a **idade** de um gambá, baseado nas demais informações fornecidas.


### a) Algumas informações possuem valores nulos, reconstrua essas informações.


In [ ]:
# Preparei meu ambiente importando as bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import scipy.stats as stats
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

# Alem de importa o dataframe chamei as primeiras linhas para compreender o dataframe
df = pd.read_csv('/content/gamba.csv')
print(df.shape)
df.head()

(104, 11)


,sex,age,hdlngth,skullw,totlngth,taill,footlgth,earconch,eye,chest,belly
0,m,8.0,94.1,60.4,89.0,36.0,74.5,54.5,15.2,28.0,36.0
1,f,6.0,92.5,57.6,91.5,36.5,72.5,51.2,16.0,28.5,33.0
2,f,6.0,94.0,60.0,95.5,39.0,75.4,51.9,15.5,30.0,34.0
3,f,6.0,93.2,57.1,92.0,38.0,76.1,52.2,15.2,28.0,34.0
4,f,2.0,91.5,56.3,85.5,36.0,71.0,53.2,15.1,28.5,33.0


In [ ]:
# Verifiquei quais são os valores que possuir valores vazio
isnull = df.isnull().sum()
print(isnull)

sex          0
age          2
hdlngth     15
skullw       0
totlngth    14
taill        0
footlgth    14
earconch     0
eye          0
chest        0
belly        0
dtype: int64


In [ ]:
# Como tem valores ausente, excluir os dados da minha coluna Y

df = df.dropna(subset=['age'])

In [ ]:
# Criei uma copia do meu dataframe pois vou realizar a padronização das colunas para dummy

df1 = df.copy()
df1['dummy_sex'] = df.apply( lambda row: 1 if row['sex']=='m' else 0 , axis=1)
df1.drop('sex', axis=1, inplace=True)

# Criei um filtro do meu dataframe para poder tirar todas colunas nulas

filtro_df1 = df1[df1['hdlngth'].notnull() & df1['totlngth'].notnull() & df1['footlgth'].notnull()]
filtro_df1.head()

,age,hdlngth,skullw,totlngth,taill,footlgth,earconch,eye,chest,belly,dummy_sex
0,8.0,94.1,60.4,89.0,36.0,74.5,54.5,15.2,28.0,36.0,1
1,6.0,92.5,57.6,91.5,36.5,72.5,51.2,16.0,28.5,33.0,0
2,6.0,94.0,60.0,95.5,39.0,75.4,51.9,15.5,30.0,34.0,0
3,6.0,93.2,57.1,92.0,38.0,76.1,52.2,15.2,28.0,34.0,0
4,2.0,91.5,56.3,85.5,36.0,71.0,53.2,15.1,28.5,33.0,0


In [ ]:
# Passei minhas colunas X e Y para criar um modelo de regressão linear

x = filtro_df1[['hdlngth','skullw','totlngth','taill','footlgth','earconch','eye','chest','belly','dummy_sex']].to_numpy()
y = filtro_df1['age'].to_numpy()
model = LinearRegression()
model.fit(x, y)
score_original = model.score(x, y).round(2)
print(score_original)

# Após imprimir o Resultado do modelo verifiquei que valor tava muito ruim

0.21


In [ ]:
# Passei minhas colunas X e Y para criar um modelo de regressão linear
# Com base no dado cabeça

x = filtro_df1[['skullw','taill','earconch','eye','chest','belly','dummy_sex']].to_numpy()
y = filtro_df1['hdlngth'].to_numpy()
mode2 = LinearRegression()
mode2.fit(x, y)
score = mode2.score(x, y).round(2)
print(score)

0.67


In [ ]:
# Após verificar, observei que o resultado da regressão estava melhor.
# Criei uma função para consultar as linhas vazias e gerar uma predição dos valores.

def completar_cabeca(row):
  if pd.isna(row['hdlngth']):
    campos = row[['skullw','taill','earconch','eye','chest','belly','dummy_sex']]

    return round(mode2.predict([campos])[0])
  return row['hdlngth']

In [ ]:
# Para garantir que não tivesse problemas com os dados realizei a copia dos dados para aplicar a função
# Verifquei que eles completou as linhas ausentes

df2 = df1.copy()
df2['nova_cabeca'] = df1.apply(completar_cabeca, axis=1)
df2.tail()

,age,hdlngth,skullw,totlngth,taill,footlgth,earconch,eye,chest,belly,dummy_sex,nova_cabeca
99,1.0,NaN,56.0,81.5,36.5,66.0,46.8,14.8,23.0,27.0,1,91.0
100,1.0,88.6,54.7,82.5,39.0,64.4,48.0,14.0,25.0,33.0,1,88.6
101,6.0,92.4,55.0,89.0,38.0,63.5,45.4,13.0,25.0,30.0,0,92.4
102,4.0,91.5,55.2,82.5,36.5,62.9,45.9,15.4,25.0,29.0,1,91.5
103,3.0,93.6,59.9,89.0,40.0,67.6,46.0,14.8,28.5,33.5,0,93.6


In [ ]:
# Vou aplicar o mesmo nas colunas tamanho total e pernas para poder gerar os dado

x = filtro_df1[['skullw','taill','earconch','eye','chest','belly','dummy_sex']].to_numpy()
y = filtro_df1['totlngth'].to_numpy()
mode3 = LinearRegression()
mode3.fit(x, y)
score = mode3.score(x, y).round(2)
print(score)

0.58


In [ ]:
# Primeiro foi criado uma função para as tamanho total

def completar_tamanho_total(row):
  if pd.isna(row['totlngth']):
    campos = row[['skullw','taill','earconch','eye','chest','belly','dummy_sex']]

    return round(mode3.predict([campos])[0])
  return row['totlngth']

In [ ]:
df3 = df2.copy()
df3['novo_tamanho_total'] = df2.apply(completar_tamanho_total, axis=1)
df3.tail()

,age,hdlngth,skullw,totlngth,taill,footlgth,earconch,eye,chest,belly,dummy_sex,nova_cabeca,novo_tamanho_total
99,1.0,NaN,56.0,81.5,36.5,66.0,46.8,14.8,23.0,27.0,1,91.0,81.5
100,1.0,88.6,54.7,82.5,39.0,64.4,48.0,14.0,25.0,33.0,1,88.6,82.5
101,6.0,92.4,55.0,89.0,38.0,63.5,45.4,13.0,25.0,30.0,0,92.4,89.0
102,4.0,91.5,55.2,82.5,36.5,62.9,45.9,15.4,25.0,29.0,1,91.5,82.5
103,3.0,93.6,59.9,89.0,40.0,67.6,46.0,14.8,28.5,33.5,0,93.6,89.0


In [ ]:
x = filtro_df1[['skullw','taill','earconch','eye','chest','belly','dummy_sex']].to_numpy()
y = filtro_df1['footlgth'].to_numpy()
mode4 = LinearRegression()
mode4.fit(x, y)
score = mode4.score(x, y).round(2)
print(score)

0.62


In [ ]:
def completar_pernas(row):
  if pd.isna(row['footlgth']):
    campos = row[['skullw','taill','earconch','eye','chest','belly','dummy_sex']]

    return round(mode4.predict([campos])[0])
  return row['footlgth']

In [ ]:
df4 = df3.copy()
df4['novo_pernas'] = df3.apply(completar_pernas, axis=1)
df4.tail()

,age,hdlngth,skullw,totlngth,taill,footlgth,earconch,eye,chest,belly,dummy_sex,nova_cabeca,novo_tamanho_total,novo_pernas
99,1.0,NaN,56.0,81.5,36.5,66.0,46.8,14.8,23.0,27.0,1,91.0,81.5,66.0
100,1.0,88.6,54.7,82.5,39.0,64.4,48.0,14.0,25.0,33.0,1,88.6,82.5,64.4
101,6.0,92.4,55.0,89.0,38.0,63.5,45.4,13.0,25.0,30.0,0,92.4,89.0,63.5
102,4.0,91.5,55.2,82.5,36.5,62.9,45.9,15.4,25.0,29.0,1,91.5,82.5,62.9
103,3.0,93.6,59.9,89.0,40.0,67.6,46.0,14.8,28.5,33.5,0,93.6,89.0,67.6


In [ ]:
# Dropei as colunas para ficar somente as colunas novas

df5 = df4.drop(['hdlngth' , 'totlngth' , 'footlgth'], axis=1)
isnull = df5.isnull().sum()
print(isnull)

age                   0
skullw                0
taill                 0
earconch              0
eye                   0
chest                 0
belly                 0
dummy_sex             0
nova_cabeca           0
novo_tamanho_total    0
novo_pernas           0
dtype: int64


### b) Avalie se a reconstrução do modelo aumentou ou diminuiu o R2 da regressão preditiva da idade do gambá


In [ ]:
# Após verificar que não tinha mais valores vazios vou criar um modelo de regressão linear
# Com os dados gerados para vericar se não existe valores vazio

x = df5.drop(columns=['age']).to_numpy()
y = df5['age'].to_numpy()
model = LinearRegression()
model.fit(x, y)
score_reconstruido = model.score(x, y).round(2)

In [ ]:
print(f"O R² original foi de: {score_original}")
print(f"Após a reconstrução o R² foi de: {score_reconstruido}")

if score_reconstruido > score_original:
  print("A reconstrução do modelo aumentou o R²")
else:
  print("A reconstrução do modelo diminuiu o R²")

O R² original foi de: 0.21
Após a reconstrução o R² foi de: 0.19
A reconstrução do modelo diminuiu o R²


## 2) Calcule uma regressão linear para determinar o comprimento do crânio do gambá, baseado nas demais informações.

### a) Divida a base de dados em treino e teste.


In [ ]:
# Dividir minha base total em teste e treino
df_train = df5.sample(60)

In [ ]:
# retiro da base principal as observações que utilizo na base de treino
df5 = df5.drop(df_train.index)

In [ ]:
# Crio uma lista com 10 amostras de 50 observações cada a partir
# das 60 observações que sobraram após a retirada das observações
# de treino. HAVERÁ REPETIÇÃO
samples = []
for i in range(10):
  samples.append(df5.sample(20))

### b) Use um modelo de bootstrap para gerar várias amostras de teste.


In [ ]:
# Criei minhas variáveis de treino
X_train = df_train.drop(columns=['skullw']).to_numpy()
Y_train = df_train['skullw'].to_numpy()

In [ ]:
# Criei meu modelo de Regressão Linear
model_linear = LinearRegression()
model_linear.fit(X_train, Y_train)
score_linear = model_linear.score(X_train, Y_train)
score_linear

0.716358635549801

In [ ]:
# Criei meu modelo de Regressão Polinomial
poly_features = PolynomialFeatures(degree=3)
X_poly_train = poly_features.fit_transform(X_train)
model_poly = LinearRegression()
model_poly.fit(X_poly_train, Y_train)
score_polynomial = model_poly.score(X_poly_train, Y_train)
score_polynomial

1.0

### c) Compare, usando um teste de hipóteses, o r² de treino e o r² de teste. Foi detectado overfit?

In [ ]:
# Começar a testar o modelo
test_score_linear = []
test_score_polynomial = []

for sample in samples:
    # Criando variáveis X, Y para regressão linear
    X_test = sample.drop('skullw', axis=1).to_numpy()
    Y_test = sample['skullw'].to_numpy()
    test_score_linear.append(model_linear.score(X_test, Y_test))

    # Criando variável polinomial para o X
    X_test_poly = poly_features.fit_transform(X_test)
    test_score_polynomial.append(model_poly.score(X_test_poly, Y_test))

print(test_score_linear)
print(test_score_polynomial)

[0.5873540985195265, 0.6378179102760388, 0.5515085495521437, 0.4705170226915949, 0.4489836338613291, 0.46664876162260494, 0.38450825677466804, 0.5465098779262305, 0.29571742667235135, 0.35624670061803787]
[-11.705661328876012, -15.555531546066, -16.579008808857836, -3.798906771704213, -6.276011942541712, -7.350670810791719, -3.4600476039364, -2.8172528297771047, -1.5385847377368398, -4.663159870033752]


In [ ]:

alpha = 0.1
# Modelagem do teste de hipóteses
# H0: score de teste é igual ao score de treino
#     a minha regressão não perde poder de explicação quando testa novas variáveis
# H1: score de teste é diferente do score de treino
#     a minha regressão não explica a variável de interesse da mesma maneira que o teste.

t_stat, p_valor = stats.ttest_1samp(test_score_linear, popmean=score_linear)
print(score_linear)
print(p_valor)

0.716358635549801
5.865842301702273e-05


In [ ]:
if p_valor < alpha:
  print("Regressão Linear: Aceito H1 e rejeito H0")
else:
  print("Regressão Linear: Aceito H0 e rejeito H1")

Regressão Linear: Aceito H1 e rejeito H0
O valor de R² regreção polimonial 1.0
O p valor foi de: 0.0008369968542097047
Regressão Polinomial: Aceito H1 e rejeito H0


In [ ]:
t_stat, p_valor = stats.ttest_1samp(test_score_polynomial, popmean=score_polynomial)
print(f'O valor de R² regreção polinomial {score_polynomial}')
print(f'O p valor foi de: {p_valor}')
if p_valor < alpha:
  print("Regressão Polinomial: Aceito H1 e rejeito H0")
else:
  print("Regressão Polinomial: Aceito H0 e rejeito H1")

O valor de R² regreção polimonial 1.0
O p valor foi de: 0.0008369968542097047
Regressão Polinomial: Aceito H1 e rejeito H0


Na minha interpretação, houve overfitting, pois o modelo polinomial apresentou um R² muito alto.